In [1]:
#import required packages
import numpy as np #this is the linear algebra package for working with matrices
import pandas as pd #pandas is a dataframe package that is useful for managing network attributes
import math #use this package to take the log of a scalar, use numpy to take the element-wise log of an array
import copy
import os

In [2]:
df = pd.read_csv('final_merged.csv',sep=",", header = None)
df.columns = ["index","Language", "Influenced By", "Influenced","Year","duplicate"]
df = df[["Language","Influenced By","Influenced","Year"]]
# whitespaces stripped for language data field
df['Language']=df['Language'].str.strip()
df

,Language,Influenced By,Influenced,Year
0,Assembly language,NaN,NaN,1947.0
1,Plankalkül,[Begriffsschrift],"[Superplan, ALGOL 58]",1948.0
2,Short Code,[ENIAC Short Code],"[Intermediate programming language, OMNIBAC Sy...",1950.0
3,G-code,NaN,NaN,1950.0
4,Superplan,[Plankalkül],[ALGOL 58],1951.0
...,...,...,...,...
689,Z notation,NaN,NaN,NaN
690,Zebra Programming Language,[ANSI BASIC],NaN,NaN
691,Zeno,NaN,NaN,NaN
692,ZOPL,NaN,NaN,NaN


In [3]:
# number of duplicates
n = len(pd.unique(df['Language']))
print(n)

672


In [4]:
# drop duplicate data
df = df.drop_duplicates(subset=['Language','Influenced By','Influenced',"Year"])
df

,Language,Influenced By,Influenced,Year
0,Assembly language,NaN,NaN,1947.0
1,Plankalkül,[Begriffsschrift],"[Superplan, ALGOL 58]",1948.0
2,Short Code,[ENIAC Short Code],"[Intermediate programming language, OMNIBAC Sy...",1950.0
3,G-code,NaN,NaN,1950.0
4,Superplan,[Plankalkül],[ALGOL 58],1951.0
...,...,...,...,...
689,Z notation,NaN,NaN,NaN
690,Zebra Programming Language,[ANSI BASIC],NaN,NaN
691,Zeno,NaN,NaN,NaN
692,ZOPL,NaN,NaN,NaN


In [5]:
def invalid_infl(node):
    return not (node in df['Language'].values)
def invalid_inflby(node):
    return not (node in df['Language'].values)

In [6]:
# drop languages that is in "Influenced By" or "Influenced" that is not a node and add valid dates to keep ordering of year
for index, row in df.iterrows():
    # check for validity of Influenced By
    if not pd.isna(row[1]):
        str1 = row[1][1:len(row[1])-1]
        arr = str1.split(", ")
        valid_inflby_arr = copy.deepcopy(arr)
        for i in range(len(arr)):
            if invalid_inflby(arr[i]):
                #valid_inflby_arr.pop(i)
                valid_inflby_arr.remove(arr[i])
        out = ", ".join(valid_inflby_arr)
        if not out:
            df.at[index,"Influenced By"] = float("nan")
        else:
            df.at[index,"Influenced By"] = "[" + out + "]"
    # check validity of Influenced
    if not pd.isna(row[2]):
        str2 = row[2][1:len(row[2])-1]
        arr = str2.split(", ")
        valid_infl_arr = copy.deepcopy(arr)
        for i in range(len(arr)):
            if invalid_inflby(arr[i]):
                #valid_infl_arr.pop(i)
                valid_infl_arr.remove(arr[i])
        out = ", ".join(valid_infl_arr)
        if not out:
            df.at[index,"Influenced"] = float("nan")
        else:
            df.at[index,"Influenced"] = "[" + out + "]"


In [7]:
for index, row in df.iterrows():
    # validate that when language A is influenced by language B, B has A in "Influenced", if not update
    if not pd.isna(row[1]):
        str1 = row[1][1:len(row[1])-1]
        arr1 = str1.split(", ")
        curr_lang = row[0]
        for i in range(len(arr1)):
            idx = df.index[df['Language']==arr1[i]].tolist()[0]
            if isinstance(df.at[idx,"Influenced"], str):
                infl_arr = df.at[idx,"Influenced"][1:len(df.at[idx,"Influenced"])-1].split(", ")
                if curr_lang not in infl_arr:
                    infl_arr.append(curr_lang)
                    new_infl = ", ".join(infl_arr)
                    df.at[idx,"Influenced"] = "["+new_infl+"]"
            else:
                df.at[idx,"Influenced"] = "["+curr_lang+"]"
for index, row in df.iterrows():
    # similar for if language A influenced language B, B has A in "Influenced By", if not update
    if not pd.isna(row[2]):
        str2 = row[2][1:len(row[2])-1]
        arr2 = str2.split(", ")
        curr_lang = row[0]
        for i in range(len(arr2)):
            idx = df.index[df['Language']==arr2[i]].tolist()[0]
            if isinstance(df.at[idx,"Influenced By"], str):
                inflby_arr = df.at[idx,"Influenced By"][1:len(df.at[idx,"Influenced By"])-1].split(", ")
                if curr_lang not in inflby_arr:
                    inflby_arr.append(curr_lang)
                    new_inflby = ", ".join(inflby_arr)
                    df.at[idx,"Influenced By"] = "["+new_inflby+"]"
            else:
                df.at[idx,"Influenced By"] = "["+curr_lang+"]"


    #gets row index of certain col value
    #df.index[df['Language']=="Scala"].tolist()[0]

In [8]:
# df with rows without year data
df_year_null = df[df['Year'].isnull()][["Language","Influenced By","Influenced"]]
df_year_null

,Language,Influenced By,Influenced
420,A-0 System,NaN,NaN
421,A++,NaN,NaN
422,ABC ALGOL,NaN,NaN
423,ACC,NaN,NaN
424,Distributed Application Specification Language,NaN,NaN
...,...,...,...
689,Z notation,NaN,NaN
690,Zebra Programming Language,NaN,NaN
691,Zeno,NaN,NaN
692,ZOPL,NaN,NaN


In [9]:
df1 = df_year_null[df_year_null['Influenced By'].notnull()]
df1

,Language,Influenced By,Influenced
430,Agora,[Self],NaN
436,Argus,[CLU],NaN
439,BETA,[Simula],[Newspeak]
441,C/AL,"[Pascal, Object Pascal]",NaN
442,Carbon,"[C++, Rust]",NaN
447,Cg,[C],NaN
450,CobolScript,[COBOL],NaN
454,Curry,[Haskell],NaN
468,EGL,[COBOL],NaN
475,F*,"[Coq, Lean, OCaml, Standard ML, ML, F Sharp]",[Standard ML]


In [10]:
df2 = df_year_null[df_year_null['Influenced'].notnull()]
df2

,Language,Influenced By,Influenced
439,BETA,[Simula],[Newspeak]
472,EXEC 2,NaN,[Rexx]
475,F*,"[Coq, Lean, OCaml, Standard ML, ML, F Sharp]",[Standard ML]
488,GAP,NaN,[TTM]
492,Gosu,[Java],[Kotlin]
503,Hope,NaN,"[Standard ML, Miranda, Mercury]"
507,Inform,NaN,[CorVision]
516,Job Control Language,NaN,[Control Language]
585,OPS5,[Lisp],[Claire]
590,Pict,[ML],[Orc]


In [11]:
# for languages that has no dates but is influenced by some language, input valid date to keep ordering
for index, row in df.iterrows():
    if pd.isna(row[3]) and not pd.isna(row[2]) and not pd.isna(row[1]):
        df.at[index,"Year"] = "2003.0"
    elif pd.isna(row[3]) and not pd.isna(row[1]):
        df.at[index,"Year"] = "2024.0"
    # for languages with no dates and not influenced by or influence (degree 0) set year as 1
    elif pd.isna(row[3]) and pd.isna(row[2]) and pd.isna(row[1]):
        df.at[index,"Year"] = "1.0"
    elif pd.isna(row[3]) and pd.isna(row[1]):
        df.at[index,"Year"] = "1.0"

In [12]:
convert_dict = {'Year': float}
df = df.astype(convert_dict)

In [13]:
# for each value in "Influenced By" and "Influenced", validate that "Influenced By" lang are older and "Influenced" lang are newer in time
for index, row in df.iterrows():
    # validate that when language A is influenced by language B, B is an older language, if not update
    if not pd.isna(row[1]):
        str1 = row[1][1:len(row[1])-1]
        arr1 = str1.split(", ")
        valid_inflby_arr = copy.deepcopy(arr1)
        curr_lang = row[0]
        idx_a = df.index[df['Language']==curr_lang].tolist()[0]
        year_a = df.at[idx_a,"Year"]
        for i in range(len(arr1)):
            idx_b = df.index[df['Language']==arr1[i]].tolist()[0]
            year_b = df.at[idx_b,"Year"]
            if year_b > year_a:
                valid_inflby_arr.remove(arr1[i])
        out = ", ".join(valid_inflby_arr)
        if not out:
            df.at[index,"Influenced By"] = float("nan")
        else:
            df.at[index,"Influenced By"] = "[" + out + "]"

for index, row in df.iterrows():
    # similar for if language A influenced language B, B is a newer language, if not update
    if not pd.isna(row[2]):
        str2 = row[2][1:len(row[2])-1]
        arr2 = str2.split(", ")
        valid_infl_arr = copy.deepcopy(arr2)
        curr_lang = row[0]
        idx_a = df.index[df['Language']==curr_lang].tolist()[0]
        year_a = df.at[idx_a,"Year"]
        for i in range(len(arr2)):
            idx_b = df.index[df['Language']==arr2[i]].tolist()[0]
            year_b = df.at[idx_b,"Year"]
            if year_b < year_a:
                valid_infl_arr.remove(arr2[i])
        out = ", ".join(valid_infl_arr)
        if not out:
            df.at[index,"Influenced"] = float("nan")
        else:
            df.at[index,"Influenced"] = "[" + out + "]"

In [14]:
#check for duplicated languages in "Influenced By" and "Influenced"
for index,row in df.iterrows():
    if not pd.isna(row[1]):
        str1 = row[1][1:len(row[1])-1]
        arr1 = str1.split(", ")
        arr1 = [*set(arr1)]
        out = ", ".join(arr1)
        if not out:
            df.at[index,"Influenced By"] = float("nan")
        else:
            df.at[index,"Influenced By"] = "[" + out + "]"
    if not pd.isna(row[2]):
        str2 = row[2][1:len(row[2])-1]
        arr2 = str2.split(", ")
        arr2 = [*set(arr2)]
        out = ", ".join(arr2)
        if not out:
            df.at[index,"Influenced"] = float("nan")
        else:
            df.at[index,"Influenced"] = "[" + out + "]"

In [15]:
df

,Language,Influenced By,Influenced,Year
0,Assembly language,NaN,[Speedcoding],1947.0
1,Plankalkül,NaN,"[ALGOL 58, Superplan]",1948.0
2,Short Code,NaN,NaN,1950.0
3,G-code,NaN,NaN,1950.0
4,Superplan,[Plankalkül],[ALGOL 58],1951.0
...,...,...,...,...
689,Z notation,NaN,NaN,1.0
690,Zebra Programming Language,NaN,NaN,1.0
691,Zeno,NaN,NaN,1.0
692,ZOPL,NaN,NaN,1.0


In [16]:
convert_dict = {'Year': float}
df = df.astype(convert_dict)
sorted_df = df.sort_values(by=['Year'], ascending=True)
sorted_df

,Language,Influenced By,Influenced,Year
693,Z++,NaN,NaN,1.0
517,JEAN,NaN,NaN,1.0
516,Job Control Language,NaN,[Control Language],1.0
515,Jess,NaN,NaN,1.0
514,JavaFX Script,NaN,NaN,1.0
...,...,...,...,...
647,SuperTalk,[HyperTalk],NaN,2024.0
648,SYMPL,[JOVIAL],NaN,2024.0
496,GRASS,[BASIC],NaN,2024.0
668,Visual FoxPro,[dBase],NaN,2024.0


In [17]:
sorted_df.to_csv('final_cleaned.csv')

In [18]:
ids = df["Language"]
df[ids.isin(ids[ids.duplicated()])].sort_values("Language")

,Language,Influenced By,Influenced,Year


In [19]:
# deleted row of 
# 613,Swift,"[C syntax, functional programming]","[Cuneiform]",2007,Swift (parallel scripting language)
# since there is also 
# 612,Swift ,"[Objective-C, Rust, Haskell, Ruby, Python, C#, CLU, D]","[Rust]",2014,Swift (programming language)
# # wikipedia page says 612 is correct so kept that